In [1]:
%pip install vinagent==0.0.6.post3 --no-deps
%pip install langchain-core langchain-openai langchain-together langgraph python-dotenv PyYAML pydantic mlflow nest_asyncio
%pip install e2b-code-interpreter


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: langchain-core in c:\users\asus\appdata\local\programs\python\python312\lib\site-packages (0.3.68)




[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if API keys are loaded
if 'OPENAI_API_KEY' in os.environ:
    print("OPENAI_API_KEY loaded successfully.")
else:
    print("Warning: OPENAI_API_KEY not found in environment.")

OPENAI_API_KEY loaded successfully.


In [2]:
from langchain_openai import ChatOpenAI

# Initialize the LLM for testing
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Architectural Proposal: BashAgent & OS Tools Integration

## 1. Executive Summary
This proposal outlines the implementation of a specialized **System Interaction Layer** for the `vinagent` framework. The goal is to provide agents with robust, safe, and platform-independent capabilities for operating system management, while maintaining strict security boundaries.

## 2. Problem Statement
General-purpose agents often struggle with precise file system manipulations and real-time terminal interactions. Existing solutions lack:
- **Granular Control**: Atomic operations for specific FS tasks.
- **Safety Rails**: Built-in protection against destructive shell commands.
- **Persona Alignment**: A dedicated identity that prioritizes system integrity and verification.

## 3. Proposed Architecture

We propose a two-tier architecture that decouples low-level capabilities from high-level agent reasoning.

### 3.1 Tier 1: Capability Layer (`os_tools.py`)
A stateless collection of primitive tools decorated with `@primary_function`. This ensures seamless discovery by the framework's `ToolManager`.

#### Tool Specification & Implementation

In [6]:
import os
import shutil
import subprocess
import webbrowser
from pathlib import Path
from typing import List, Optional, Union, Dict, Any
from vinagent.register import primary_function

# Safety constraints
DANGEROUS_COMMANDS = [
    "rm -rf /",
    "rm -rf *",
    ":(){ :|:& };:",
    "mkfs",
    "> /dev/",
    "dd if=",
    "chmod -R 777",
    "chown -R",
    "curl",
    "wget",
    "nc -e",
    "sh -i",
    "bash -i",
    "python -c",
    "perl -e",
    "ruby -e",
    "socat",
    "poweroff",
    "reboot",
    "shutdown",
    "init 0",
    "init 6",
    "/dev/sda",
    "/dev/nvme",
]

@primary_function
def create_file(file_path: str, content: str = "") -> str:
    """
    Creates a new file at the specified path with optional content.

    Args:
        file_path (str): The path where the file should be created.
        content (str, optional): The initial content to write to the file. Defaults to "".

    Returns:
        str: A success message if the file was created, or an error message otherwise.
    """
    try:
        path = Path(file_path)
        path.parent.mkdir(parents=True, exist_ok=True)
        path.write_text(content, encoding="utf-8")
        return f"Successfully created file: {file_path}"
    except Exception as e:
        return f"Error creating file {file_path}: {str(e)}"

@primary_function
def read_file(file_path: str) -> str:
    """
    Reads the content of the file at the specified path.

    Args:
        file_path (str): The path to the file to be read.

    Returns:
        str: The content of the file if successful, or an error message otherwise.
    """
    try:
        path = Path(file_path)
        if not path.exists():
            return f"Error: File not found at {file_path}"
        if not path.is_file():
            return f"Error: {file_path} is not a file"
        return path.read_text(encoding="utf-8")
    except Exception as e:
        return f"Error reading file {file_path}: {str(e)}"

@primary_function
def update_file_content(file_path: str, content: str, mode: str = "append") -> str:
    """
    Updates the content of a file by either appending to it or overwriting it.

    Args:
        file_path (str): The path to the file to be updated.
        content (str): The content to write to the file.
        mode (str, optional): The update mode. Must be 'append' or 'overwrite'. Defaults to "append".

    Returns:
        str: A success message if the file was updated, or an error message otherwise.
    """
    try:
        path = Path(file_path)
        if not path.exists():
            return f"Error: File not found at {file_path}"
        
        if mode == "append":
            with open(path, "a", encoding="utf-8") as f:
                f.write(content)
        elif mode == "overwrite":
            path.write_text(content, encoding="utf-8")
        else:
            return f"Error: Invalid mode '{mode}'. Use 'append' or 'overwrite'."
        
        return f"Successfully updated file: {file_path} (mode: {mode})"
    except Exception as e:
        return f"Error updating file {file_path}: {str(e)}"

@primary_function
def delete_file(file_path: str) -> str:
    """
    Deletes the file at the specified path.

    Args:
        file_path (str): The path to the file to be deleted.

    Returns:
        str: A success message if the file was deleted, or an error message otherwise.
    """
    try:
        path = Path(file_path)
        if not path.exists():
            return f"Error: File not found at {file_path}"
        if not path.is_file():
            return f"Error: {file_path} is not a file"
        path.unlink()
        return f"Successfully deleted file: {file_path}"
    except Exception as e:
        return f"Error deleting file {file_path}: {str(e)}"

@primary_function
def copy_file(source_path: str, destination_path: str) -> str:
    """
    Copies a file from the source path to the destination path.

    Args:
        source_path (str): The path to the source file.
        destination_path (str): The destination path (file or directory).

    Returns:
        str: A success message if the file was copied, or an error message otherwise.
    """
    try:
        shutil.copy2(source_path, destination_path)
        return f"Successfully copied {source_path} to {destination_path}"
    except Exception as e:
        return f"Error copying file from {source_path} to {destination_path}: {str(e)}"

@primary_function
def move_file(source_path: str, destination_path: str) -> str:
    """
    Moves or renames a file from the source path to the destination path.

    Args:
        source_path (str): The path to the source file.
        destination_path (str): The destination path (file or directory).

    Returns:
        str: A success message if the file was moved, or an error message otherwise.
    """
    try:
        shutil.move(source_path, destination_path)
        return f"Successfully moved {source_path} to {destination_path}"
    except Exception as e:
        return f"Error moving file from {source_path} to {destination_path}: {str(e)}"

@primary_function
def create_folder(folder_path: str) -> str:
    """
    Creates a new folder at the specified path, including any necessary parent folders.

    Args:
        folder_path (str): The path of the folder to be created.

    Returns:
        str: A success message if the folder was created, or an error message otherwise.
    """
    try:
        Path(folder_path).mkdir(parents=True, exist_ok=True)
        return f"Successfully created folder: {folder_path}"
    except Exception as e:
        return f"Error creating folder {folder_path}: {str(e)}"

@primary_function
def delete_folder(folder_path: str) -> str:
    """
    Deletes a folder and all of its contents.

    Args:
        folder_path (str): The path of the folder to be deleted.

    Returns:
        str: A success message if the folder was deleted, or an error message otherwise.
    """
    try:
        path = Path(folder_path)
        if not path.exists():
            return f"Error: Folder not found at {folder_path}"
        if not path.is_dir():
            return f"Error: {folder_path} is not a folder"
        shutil.rmtree(path)
        return f"Successfully deleted folder: {folder_path}"
    except Exception as e:
        return f"Error deleting folder {folder_path}: {str(e)}"

@primary_function
def list_directory(directory_path: str = ".") -> Union[List[str], str]:
    """
    Lists the contents of a directory.

    Args:
        directory_path (str, optional): The path to the directory to list. Defaults to ".".

    Returns:
        Union[List[str], str]: A list of file and folder names if successful, or an error message otherwise.
    """
    try:
        path = Path(directory_path)
        if not path.exists():
            return f"Error: Directory not found at {directory_path}"
        if not path.is_dir():
            return f"Error: {directory_path} is not a directory"
        return [item.name for item in path.iterdir()]
    except Exception as e:
        return f"Error listing directory {directory_path}: {str(e)}"

@primary_function
def open_browser(url: str) -> str:
    """
    Opens the default web browser to the specified URL.

    Args:
        url (str): The URL to open in the browser.

    Returns:
        str: A success message if the browser was opened, or an error message otherwise.
    """
    try:
        webbrowser.open(url)
        return f"Successfully opened browser to: {url}"
    except Exception as e:
        return f"Error opening browser to {url}: {str(e)}"

@primary_function
def execute_bash(command: str, timeout: int = 30) -> str:
    """
    Executes a shell command and returns the output. Safety checks are performed to block dangerous commands.
    Every terminal command is wrapped in a subprocess with a strict 30-second timeout.
    
    IMPORTANT: The agent is explicitly instructed to perform pre-verification of the command
    to reduce the risk of unintended side effects.

    Args:
        command (str): The shell command to execute.
        timeout (int, optional): The maximum time in seconds to wait for the command to complete. Defaults to 30.

    Returns:
        str: The standard output and standard error of the command, or an error message if the command was blocked or failed.
    """
    # Safety Check
    cmd_lower = command.lower()
    for dangerous in DANGEROUS_COMMANDS:
        if dangerous in cmd_lower:
            return f"Error: Command blocked for safety reasons. '{dangerous}' is forbidden."

    try:
        result = subprocess.run(
            command,
            shell=True,
            text=True,
            capture_output=True,
            timeout=timeout
        )
        output = result.stdout if result.stdout else ""
        error = result.stderr if result.stderr else ""
        
        if result.returncode == 0:
            return output if output else "Command executed successfully with no output."
        else:
            return f"Command failed with return code {result.returncode}.\nOutput: {output}\nError: {error}"
            
    except subprocess.TimeoutExpired:
        return f"Error: Command execution timed out after {timeout} seconds."
    except Exception as e:
        return f"Error executing command: {str(e)}"

### 3.2 Tier 2: Cognitive Layer (`bash_agent.py`)
A specialized agent class that inherits from the base `Agent`. It is pre-configured with the `os_tools` and a specific system prompt focused on **Diligence** and **Dignity**.

#### Implementation Detail

In [7]:
from vinagent.agent.agent import Agent
from vinagent.guardrail import GuardrailManager

# Initialize the Agent directly
agent = Agent(
    llm=llm,
    description="You are a Diligent and Secure System Engineer.",
    instruction=(
        "Handle all system operations with extreme care and diligence. "
        "PERFORM PRE-VERIFICATION: Before executing any tool, especially shell commands, "
        "think step-by-step about potential side effects and verify that the command "
        "does not contain sensitive or destructive patterns."
    ),
    skills=[
        "Comprehensive file system management",
        "Secure execution of shell commands",
    ],
    num_buffered_messages=30,  # Workaround for history pollution loop
    guardrail_manager=GuardrailManager("guardrail.yaml")
)

# Register OS tools manually
local_tools = [
    create_file, read_file, update_file_content, delete_file, 
    copy_file, move_file, create_folder, delete_folder, 
    list_directory, open_browser, execute_bash
]
for tool in local_tools:
    agent.tools_manager.register_function_tool(tool)

INFO:vinagent.register.tool:Registered tool: create_file (runtime)
INFO:vinagent.register.tool:Registered tool: read_file (runtime)
INFO:vinagent.register.tool:Registered tool: update_file_content (runtime)
INFO:vinagent.register.tool:Registered tool: delete_file (runtime)
INFO:vinagent.register.tool:Registered tool: copy_file (runtime)
INFO:vinagent.register.tool:Registered tool: move_file (runtime)
INFO:vinagent.register.tool:Registered tool: create_folder (runtime)
INFO:vinagent.register.tool:Registered tool: delete_folder (runtime)
INFO:vinagent.register.tool:Registered tool: list_directory (runtime)
INFO:vinagent.register.tool:Registered tool: open_browser (runtime)
INFO:vinagent.register.tool:Registered tool: execute_bash (runtime)


## 4. Security & Safety Analysis

The proposed architecture implements security at multiple levels:
1. **Lexical Filtering**: `execute_bash` proactively blocks forbidden patterns.
2. **Process Isolation**: Every terminal command is wrapped in a subprocess with a strict **30-second timeout**.
3. **Persona-Level Guardrails**: The agent is explicitly instructed to perform pre-verification, reducing the risk of unintended side effects.
4. **Error Masking**: Raw OS exceptions are caught and converted into string feedback for the agent, preventing framework-level crashes.

### 4.1 Persona-Level Guardrails with GuardrailManager

To facilitate management of guardrail layers for a certain agent over all layers: `input, output, and tools`, we offer yaml template that you can define each guardrail class in each layers with its relevant parameters like:

In [3]:
%%writefile guardrail.yaml
guardrails:
  input:
    - name: PIIGuardrail
    - name: ScopeGuardrail
      params:
        agent_scope: ["Deeply analyzing financial markets", "System engineering and file management operations"]
    - name: ToxicityGuardrail
    - name: PromptInjectionGuardrail

  tools:
    execute_bash: 
      - name: ToxicityGuardrail # Used to enable DecisionModel validation for bash arguments
    list_directory: [] # Key exists to avoid KeyError
    read_file: []     # Key exists to avoid KeyError
    write_file: []    # Key exists to avoid KeyError
    delete_file: []   # Key exists to avoid KeyError

  output:
    - name: OutputPIIGuardrail
    - name: HallucinationGuardrail

Overwriting guardrail.yaml


### 5.1 SecureAgent Implementation (Standard Inheritance)

Following the principle of library integrity, we extend the `Agent` class using standard Python inheritance. This `SecureAgent` class introduces:
1. **Resilience**: Protection against `KeyError` and `IndexError` in the library's `GuardrailManager`.
2. **Safety**: Support for `None` tool names to prevent framework-level crashes.
3. **Decision Model Integration**: Enabling LLM-powered validation for tool arguments (like bash commands) by bridging the gap in the library's fixed-signature guardrail call.

In [8]:
import inspect
import json
from vinagent.agent.agent import Agent

class SecureAgent(Agent):
    """
    A specialized version of the Agent that provides robust guardrail handling
    and enables DecisionModel validation for tool arguments without modifying the core library.
    """
    def check_tool_guardrail(self, tool_name: str):
        # 1. Handle None tool_name (Safety check)
        if tool_name is None:
            return True
        
        # 2. Extract tool arguments from the calling frame (Surgery to avoid re-writing invoke)
        # This allows us to access the 'tool_call' local variable in the library's invoke method
        frame = inspect.currentframe().f_back
        tool_call = frame.f_locals.get('tool_call', {})
        arguments = tool_call.get('arguments', {})

        if self.guardrail_manager:
            # 3. Robust check for YAML configuration existence (KeyError protection)
            tool_grs = self.guardrail_manager.tool_guardrails.get(tool_name, [])
            
            # 4. Integrate 'Decision Model' for execute_bash (LLM-powered validation)
            # If there are guardrails configured for execute_bash, we use the DecisionModel logic
            if tool_name == "execute_bash" and arguments and tool_grs:
                # Dynamically build a decision model using the library's manager
                DecisionModel = self.guardrail_manager.add_guardrails(tool_grs)
                
                # Use the LLM to decide if the specific bash command is safe
                # We pass the full arguments dict as context for the LLM
                decision = DecisionModel.validate(self.llm, str(arguments))
                
                if not decision.allowed:
                    raise ValueError(f"[SecureGuardrail Blocked] {decision.reason}")
                return True
            
            # 5. Fallback for other tools
            if tool_grs:
                try:
                    return super().check_tool_guardrail(tool_name)
                except (KeyError, IndexError):
                    return True # Fail-safe against library internal crashes
                    
        return True

# Initialize the SecureAgent
secure_agent = SecureAgent(
    llm=llm,
    description="You are a Diligent and Secure System Engineer.",
    instruction="Handle all system operations with extreme care and diligence.",
    skills=["Secure execution of shell commands"],
    num_buffered_messages=30,
    guardrail_manager=GuardrailManager("guardrail.yaml")
)

# Register tools with the secure agent
# Ensure local_tools is defined in the previous cells
for tool in local_tools:
    secure_agent.tools_manager.register_function_tool(tool)

# Re-assign to the global agent variable so subsequent cells can use it
agent = secure_agent

print("SecureAgent initialized successfully with DecisionModel support!")

INFO:vinagent.register.tool:Registered tool: create_file (runtime)


INFO:vinagent.register.tool:Registered tool: read_file (runtime)
INFO:vinagent.register.tool:Registered tool: update_file_content (runtime)
INFO:vinagent.register.tool:Registered tool: delete_file (runtime)
INFO:vinagent.register.tool:Registered tool: copy_file (runtime)
INFO:vinagent.register.tool:Registered tool: move_file (runtime)
INFO:vinagent.register.tool:Registered tool: create_folder (runtime)
INFO:vinagent.register.tool:Registered tool: delete_folder (runtime)
INFO:vinagent.register.tool:Registered tool: list_directory (runtime)
INFO:vinagent.register.tool:Registered tool: open_browser (runtime)
INFO:vinagent.register.tool:Registered tool: execute_bash (runtime)


SecureAgent initialized successfully with DecisionModel support!


## 6. Detailed Tool Examples

This section provides standalone examples for each tool available to the agent. These examples demonstrate direct programmatic usage of the tool functions.

### 6.1 `create_file`
Creates a file with specified content. It also creates parent directories if they don't exist.

In [ ]:
result = create_file("examples/test_file.txt", "Hello from vinagent tools!")
print(result)

Successfully created file: examples/test_file.txt


### 6.2 `read_file`
Reads and returns the content of a file.

In [ ]:
content = read_file("examples/test_file.txt")
print(f"File Content:\n{content}")

File Content:
Hello from vinagent tools!


### 6.3 `update_file_content`
Appends to or overwrites an existing file.

In [ ]:
# Append content
append_result = update_file_content("examples/test_file.txt", "\nAdding more content.", mode="append")
print(append_result)

# Read back to verify
print(read_file("examples/test_file.txt"))

Successfully updated file: examples/test_file.txt (mode: append)
Hello from vinagent tools!
Adding more content.


### 6.4 `list_directory`
Lists all files and folders in a specified directory.

In [ ]:
items = list_directory("examples")
print(f"Items in 'examples': {items}")

Items in 'examples': ['test_file.txt']


### 6.5 `copy_file`
Copies a file to a new location.

In [ ]:
copy_result = copy_file("examples/test_file.txt", "examples/test_file_copy.txt")
print(copy_result)

Successfully copied examples/test_file.txt to examples/test_file_copy.txt


### 6.6 `move_file`
Moves or renames a file.

In [ ]:
move_result = move_file("examples/test_file_copy.txt", "examples/renamed_file.txt")
print(move_result)

Successfully moved examples/test_file_copy.txt to examples/renamed_file.txt


### 6.7 `create_folder`
Creates a new directory (recursively).

In [ ]:
folder_result = create_folder("examples/nested/folder/structure")
print(folder_result)

Successfully created folder: examples/nested/folder/structure


### 6.8 `execute_bash`
Executes a shell command. Includes safety checks for dangerous commands.

In [ ]:
# A safe command like 'echo' or 'ls' (dir on Windows)
bash_result = execute_bash("echo 'Testing bash execution from vinagent tools'")
print(f"Output:\n{bash_result}")

Output:
'Testing bash execution from vinagent tools'



### 6.9 `open_browser`
Opens a URL in the default system browser.

In [ ]:
browser_result = open_browser("https://github.com/datascienceworld-kan/vinagent")
print(browser_result)

Successfully opened browser to: https://github.com/datascienceworld-kan/vinagent


### 6.10 `delete_file`
Permanently removes a file.

In [ ]:
del_file_result = delete_file("examples/renamed_file.txt")
print(del_file_result)

Successfully deleted file: examples/renamed_file.txt


### 6.11 `delete_folder`
Removes a folder and all its contents.

In [ ]:
del_folder_result = delete_folder("examples")
print(del_folder_result)

Successfully deleted folder: examples


## 7. Operational Demonstration

The following cell demonstrates the `BashAgent` in action. It will use the refined tools and security logic defined above to perform a simple system engineering task.

In [ ]:
# Execute a task that involves directory and file management
task = """
1. Check if a directory named 'bash_agent_demo' exists. 
2. If it does not exist, create it.
3. Inside that directory, create a file named 'hello_world.txt' with a friendly message.
"""

print(f"Executing Task: {task}")
response = agent.invoke(task)

if hasattr(response, 'content'):
    print("\nAgent Final Response:\n", response.content)
else:
    print("\nAgent Final Response:\n", response)

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user


Executing Task: 
1. Check if a directory named 'bash_agent_demo' exists. 
2. If it does not exist, create it.
3. Inside that directory, create a file named 'hello_world.txt' with a friendly message.



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 1 iterations.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Agent Final Response:
 The directory named 'bash_agent_demo' has been successfully created, and inside that directory, the file 'hello_world.txt' has been created with the message: "Hello, World! Welcome to the bash agent demo."


## 8. Agent-Based Tool Strategy

This section demonstrates how the agent naturally uses these tools when given high-level natural language instructions. The agent reasons about which tool to call and with what arguments.

### 8.1 File Creation & Writing
Instructing the agent to create a specific file with content.

In [ ]:
agent.invoke("Please create a file named 'agent_note.txt' in the current directory with the text 'This was created by the agent.'")

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'create_file', 'tool_type': 'function', 'arguments': {'file_path': 'agent_note.txt', 'content': 'This was created by the agent.'}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool create_file({'file_path': 'agent_note.txt', 'content': 'This was created by the agent.'})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:httpx:HTTP Request:

AIMessage(content="The file named 'agent_note.txt' has been successfully created in the current directory with the text 'This was created by the agent.'", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 11587, 'total_tokens': 11614, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 6912}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--1c84f421-b032-41c3-90ca-cbdad33319ca-0', usage_metadata={'input_tokens': 11587, 'output_tokens': 27, 'total_tokens': 11614, 'input_token_details': {'audio': 0, 'cache_read': 6912}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 8.2 File Reading & Information Retrieval
Asking the agent to read a file and provide the information within.

In [ ]:
response = agent.invoke("Read the file 'agent_note.txt' and tell me what is written inside.")
print(response.content if hasattr(response, 'content') else response)

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'read_file', 'tool_type': 'function', 'arguments': {'file_path': 'agent_note.txt'}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool read_file({'file_path': 'agent_note.txt'})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The content of the file 'agent_note.txt' is: "This was created by the agent."


### 8.3 Content Update & Appending
Asking the agent to modify or update existing files.

In [ ]:
agent.invoke("Append a line saying 'Update: Version 2' to the file 'agent_note.txt'.")

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'update_file_content', 'tool_type': 'function', 'arguments': {'file_path': 'agent_note.txt', 'content': 'Update: Version 2', 'mode': 'append'}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool update_file_content({'file_path': 'agent_note.txt', 'content': 'Update: Version 2', 'mode': 'append'})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'read_file', 'too

AIMessage(content='This was created by the agent. Update: Version 2', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 20837, 'total_tokens': 20849, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--d93f6131-06df-4fb8-a782-d3ed53afb4f9-0', usage_metadata={'input_tokens': 20837, 'output_tokens': 12, 'total_tokens': 20849, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 8.4 Directory Listing & Exploration
Requesting the agent to explore the workspace.

In [ ]:
agent.invoke("List the files in the current directory so I can see if 'agent_note.txt' exists.")

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'list_directory', 'tool_type': 'function', 'arguments': {'directory_path': '.'}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool list_directory({'directory_path': '.'})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="The current directory contains the following files and folders:\n- agent_note.txt\n- bash_agent_demo\n- bash_agent_guide.ipynb\n- compare_vinagent_with_langgraph.ipynb\n- financial_usercases\n- guardrail.yaml\n- templates\n\nYou can confirm that 'agent_note.txt' exists in this directory.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 22880, 'total_tokens': 22948, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--03855d23-c2ed-4084-a40d-e374952221c9-0', usage_metadata={'input_tokens': 22880, 'output_tokens': 68, 'total_tokens': 22948, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi

### 8.5 File Management (Copy/Move/Rename)
Asking the agent to organize files.

In [ ]:
agent.invoke("Make a copy of 'agent_note.txt' called 'agent_note_backup.txt'.")

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'copy_file', 'tool_type': 'function', 'arguments': {'source_path': 'agent_note.txt', 'destination_path': 'agent_note_backup.txt'}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool copy_file({'source_path': 'agent_note.txt', 'destination_path': 'agent_note_backup.txt'})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:httpx:HTTP Request:

AIMessage(content="The copy of 'agent_note.txt' has been successfully created as 'agent_note_backup.txt'.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 22769, 'total_tokens': 22788, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--e9a32114-d116-4047-9356-48f982077e92-0', usage_metadata={'input_tokens': 22769, 'output_tokens': 19, 'total_tokens': 22788, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 8.6 Shell Execution
Using the agent to run complex or system-level checks.

In [ ]:
agent.invoke("Use the bash tool to check the current working directory path.")

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'execute_bash', 'tool_type': 'function', 'arguments': {'command': 'pwd', 'timeout': 30}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool execute_bash({'command': 'pwd', 'timeout': 30})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'execute_bash', 'tool_type': 'function', 'arguments': {'command': 'cd', 'timeout': 30}, 'module_path': '__runtime__'}
INFO:vina

AIMessage(content='The current working directory path is `d:\\vinagent\\cookbook`.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 20781, 'total_tokens': 20796, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--1a861782-8b6f-481a-b8ec-30e8b6879a18-0', usage_metadata={'input_tokens': 20781, 'output_tokens': 15, 'total_tokens': 20796, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 8.7 Web Interaction
Instructing the agent to open documentation or relevant links.

In [ ]:
agent.invoke("Open the Google homepage in my browser.")

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'open_browser', 'tool_type': 'function', 'arguments': {'url': 'https://www.google.com'}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool open_browser({'url': 'https://www.google.com'})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='The Google homepage has been successfully opened in your browser. If you need further assistance or information, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 22795, 'total_tokens': 22819, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_4ea526dd98', 'finish_reason': 'stop', 'logprobs': None}, id='run--146f2389-af43-4633-a995-1a022e2e36bd-0', usage_metadata={'input_tokens': 22795, 'output_tokens': 24, 'total_tokens': 22819, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 8.8 Cleanup Operations
Asking the agent to remove temporary files or directories.

In [ ]:
agent.invoke("Clean up by deleting 'agent_note.txt' and 'agent_note_backup.txt'.")

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'delete_file', 'tool_type': 'function', 'arguments': {'file_path': 'agent_note.txt'}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool delete_file({'file_path': 'agent_note.txt'})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'delete_file', 'tool_type': 'function', 'arguments': {'file_path': 'agent_note_backup.txt'}, 'module_path': '__runtime__'}
INFO:vinag

AIMessage(content='All specified files have been successfully deleted. If you need any further assistance, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 20646, 'total_tokens': 20666, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--7fc48fe4-8d5a-4250-8dd2-78284992960b-0', usage_metadata={'input_tokens': 20646, 'output_tokens': 20, 'total_tokens': 20666, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 9. Advanced Security: Customized Guardrails

This section demonstrates how to define a customized guardrail programmatically and integrate it directly into the Agent's input processing layer. This is useful for enforcing domain-specific rules or protecting sensitive organizational data.

### 9.1 Defining a Customized Guardrail
We create a class that inherits from `GuardRailBase` and define our specific security rules.

In [ ]:
from vinagent.guardrail import GuardRailBase, GuardrailDecision

class SystemSecurityGuardrail(GuardRailBase):
    name: str = "system_security_check"

    def prompt_section(self) -> str:
        """Define the security rules for this guardrail."""
        return """
SYSTEM SECURITY RULES
Detect whether the input includes any of the following sensitive information:
- Email addresses
- Phone numbers
- National identity numbers
- Internal server IP addresses
"""

    def result_field(self) -> str:
        """Return the unique identifier for this guardrail's result."""
        return "system_security_check"

### 9.2 Initializing the Guardrail Model
We combine our custom guardrail with standard ones into a `GuardrailDecision` model.

In [ ]:
from vinagent.guardrail import PIIGuardrail, ToxicityGuardrail, PromptInjectionGuardrail

# Create the decision model for input validation
InputDecisionModel = GuardrailDecision.add_guardrails(
    [
        SystemSecurityGuardrail(),
        PIIGuardrail(),
        ToxicityGuardrail(),
        PromptInjectionGuardrail()
    ]
)

### 9.3 Integrating with the Agent
When initializing the `Agent`, we pass the decision model to the `input_guardrail` parameter.

In [ ]:
from vinagent.agent.agent import Agent

# Initialize a secure agent with the custom input guardrail
secure_agent = Agent(
    llm=llm,
    description="You are a Secure System Engineer.",
    instruction="Perform system tasks while ensuring no sensitive data is leaked or processed.",
    input_guardrail=InputDecisionModel
)

print("Secure Agent initialized with customized guardrails.")

Secure Agent initialized with customized guardrails.


### 9.4 Testing Guardrail Enforcement
Now we test the agent with an input that violates our custom security rules. The guardrail should block the request before it reaches the LLM.

In [ ]:
test_input = "Please send a log summary to admin@company.internal regarding the server at 192.168.1.50."
print(f"Testing input: {test_input}")

try:
    response = secure_agent.invoke(test_input)
    print("Agent Response:", response.content if hasattr(response, 'content') else response)
except Exception as e:
    print(f"\nGuardrail Blocked the request!\nError: {str(e)}")

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user


Testing input: Please send a log summary to admin@company.internal regarding the server at 192.168.1.50.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Guardrail Blocked the request!
Error: Input contains sensitive information including an email address and an internal server IP address.


## 10. Cloud Sandbox: E2B Code Interpreter Integration

This section demonstrates how to integrate the **E2B Code Interpreter** structure directly into the `vinagent` Agent. Unlike local tools, this provides a secure, stateful Python kernel in the cloud.

### 10.1 Defining the Code Interpreter Tool
We follow the E2B structure using `Sandbox.create()` and `sandbox.run_code()`.

In [7]:
from e2b_code_interpreter import Sandbox
from vinagent.register import primary_function

@primary_function
def code_interpreter_e2b(code: str) -> str:
    """
    Execute python code in a stateful E2B Cloud Sandbox and return the result.
    Use this for complex calculations, data analysis, or file processing in a secure environment.
    
    Args:
        code (str): The Python code to execute.
    """
    try:
        # Following the Sandbox.create() pattern
        with Sandbox.create() as sandbox:
            execution = sandbox.run_code(code)
            
            # Collect results (logs, texts, errors)
            result = ""
            if execution.text:
                result += execution.text
            if execution.error:
                result += f"\nExecution Error: {execution.error.name}: {execution.error.value}\n{execution.error.traceback}"
            
            return result if result else "Code executed successfully with no output."
    except Exception as e:
        return f"Cloud Sandbox Error: {str(e)}"

### 10.2 Registering with the Agent
The `vinagent` Agent automatically handles the `tool_calls` loop and message history, so you don't need to manually manage the `messages` list as shown in the raw SDK examples.

In [8]:
# Swap or add the cloud tool
agent.tools_manager.register_function_tool(code_interpreter_e2b)
print("E2B Code Interpreter registered successfully!")

INFO:vinagent.register.tool:Registered tool: code_interpreter_e2b (runtime)


E2B Code Interpreter registered successfully!


### 10.3 Demonstration: Strawberry Test
Let's test if the agent can solve the 'strawberry' problem using the Cloud Sandbox.

In [9]:
task = "How many r's are in the word 'strawberry'? Use the E2B code interpreter to get a guaranteed result."
print(f"Task: {task}")

try:
    # The agent will now automatically call code_interpreter_e2b with the correct Python code
    response = agent.invoke(task)
    print("\nAgent Result:\n", response.content if hasattr(response, 'content') else response)
except Exception as e:
    print(f"Deployment error: {str(e)}")

INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user


Task: How many r's are in the word 'strawberry'? Use the E2B code interpreter to get a guaranteed result.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Tool calling iteration 1/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'code_interpreter_e2b', 'tool_type': 'function', 'arguments': {'code': "word = 'strawberry'\ncount_r = word.count('r')\ncount_r"}, 'module_path': '__runtime__'}
INFO:e2b.api:Request POST https://api.e2b.app/sandboxes
INFO:e2b.api.client_sync:Request: POST https://api.e2b.app/sandboxes
INFO:e2b.api.client_sync:Response: 201 https://api.e2b.app/sandboxes
INFO:httpx:HTTP Request: POST https://api.e2b.app/sandboxes "HTTP/1.1 201 Created"
INFO:e2b.api:Response 201
INFO:e2b.api.client_sync:Request: POST https://49999-icxxtg9ublwu8ezrln1qk.e2b.app/execute
INFO:e2b.api.client_sync:Response: 200 https://49999-icxxtg9ublwu8ezrln1qk.e2b.app/execute
INFO:httpx:HTTP Request: POST https://49999-icxxtg9ublwu8


Agent Result:
 The word 'strawberry' contains 3 occurrences of the letter 'r'.
